In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
fin_report = pd.read_csv("fin_report.csv")

In [3]:
#定義策略 → 預期股利率 >= 4% 且 EPS同比成長 >= 6% 且 TTM PE <= 15才進場，只要下期不符合就出場
def basic_analysis_strategy(data):
    data.reset_index(inplace = True)
    data["position"] = np.zeros(len(data))
    has_position = False
    for i in range(4, len(data)):
        if ((data["Expeceted_Dividend_Rate"][i] >= 0.04) and (data["EPS_growth"][i] >= 0.06) and (data["TTM_PE"][i] <= 15)):
            if not has_position:
                data["position"].iloc[i] = 1
                has_position = True
        elif ((data["Expeceted_Dividend_Rate"][i] <= 0.04) or (data["EPS_growth"][i] <= 0.06) or (data["TTM_PE"][i] > 15)):
            if has_position:
                data["position"].iloc[i] = -1
                has_position = False
    return(data["position"])

#以下跑策略找進出場點
temp_result = fin_report.groupby("Code", sort = False, group_keys = False).apply(basic_analysis_strategy)

result = []
for i in range(len(temp_result)):
    result = np.append(result, temp_result.iloc[i, :])
    
fin_report["Position"] = result

In [4]:
stockprice = pd.read_excel("stockprice.xlsx")
stockprice.columns = ["Company", "Date", "Code", "Close"]
stockprice["Date"] = pd.to_datetime(stockprice["Date"])

In [5]:
#建立函數創造季節的欄位，方便等等merge
def create_season_id(x):
    if(x.month == 5):
        return(1)
    elif(x.month == 8):
        return(2)
    elif(x.month == 11):
        return(3)
    elif(x.month == 3):
        return(4)

#執行函數
stockprice["Season"] = stockprice.Date.apply(create_season_id)

In [6]:
#建立函數創造年的欄位，方便等等merge
def create_year_id(x):
    if (x.month == 3):
        return(x.year - 1)
    else:
        return(x.year)
    
#執行函數
stockprice["Year"] = stockprice.Date.apply(create_year_id)

In [7]:
stockprice = stockprice[["Company", "Code", "Year", "Season", "Date", "Close"]]

consider_list = stockprice.Code.unique()[stockprice.groupby("Code", sort = False).apply(lambda x : len(x)) == 17]
stockprice = stockprice[stockprice.Code.isin(consider_list)]

In [8]:
#合併股價與財報
temp_result = pd.merge(fin_report, stockprice, on = ["Code", "Year", "Season"])

In [9]:
temp_result = temp_result[["Company_x", "Industry", "Code", "Year", "Season", "announce_day", "Date", "TTM_PE", "EPS_growth", "Expeceted_Dividend_Rate", "Position", "Close"]]
temp_result.columns = ["Company", "Industry", "Code", "Year", "Season", "announce_day", "Date", "TTM_PE", "EPS_growth", "Expeceted_Dividend_Rate", "Position", "Close"]

In [10]:
def decide_in_or_out(x):
    x.reset_index(inplace = True)
    if(x.Position.cumsum()[len(x) - 1] == 1):
        if(x.Position[len(x) - 1] == 1):
             return(np.append(x.Position[:(len(x) - 1)], 0))
        elif(x.Position[len(x) - 1] == 0):
            return(np.append(x.Position[:(len(x) - 1)], -1))
    else:
        return(np.array(x.Position[:(len(x))]))    

In [11]:
temp = temp_result.groupby("Code", sort = False, as_index = False).apply(decide_in_or_out)

result = []
for i in range(len(temp)):
    result = np.append(result, temp[i])
    
temp_result["Position_new"] = result

In [12]:
#定義函數檢查最後一天進出場狀況是否有問題
def check(x):
    x.reset_index(inplace = True)
    return(x["Position_new"].cumsum()[len(x)-1])

temp_result.Code.unique()[temp_result.groupby("Code",sort = False).apply(check) != 0]

array([], dtype=int64)

In [13]:
#建立函數找出交易日
def get_trade_data(x):
    return(x[x.Position_new != 0])

#執行函數
result = temp_result.groupby("Code", as_index = False, group_keys = False).apply(get_trade_data).reset_index(drop = True)

In [14]:
#整理進場資料
result_in = result.iloc[range(0,len(result), 2), :]

result_in.reset_index(inplace = True, drop = True)

#整理出場資料
result_out = result.iloc[range(1,len(result), 2), :]

result_out.drop(['Company', 'Industry', 'Year', 'Season', 'announce_day', 'TTM_PE',
                 'EPS_growth', 'Expeceted_Dividend_Rate', 'Position','Position_new'], axis = 1, inplace = True)

result_out.reset_index(inplace = True, drop = True)
result_out.columns = ["Code_out", "Date_out", "Close_out"]

#合併進出場資料
result = pd.concat([result_in, result_out], axis = 1)

C:\Users\andy5\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [15]:
#計算報酬與持有天數
result["Return"] = (result["Close_out"] /  result["Close"]) - 1
result["holding_days"] = (result["Date_out"] - result["Date"]).apply(lambda x : x.days)
result["Daily_Return_geometry"] = ((result["Close_out"] /  result["Close"]) ** (1 / result["holding_days"])) - 1
result["Daily_Return_arithmetic"] = ((result["Close_out"] / result["Close"]) - 1) / result["holding_days"]

In [16]:
result.to_csv("Result.csv", index = False)